In [6]:
# -*- coding:utf-8 -*-
# Parsing dividends data from DART

import urllib.request
import urllib.parse
import xlsxwriter
import os
import time
import sys
import getopt
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
import parser
import pandas as pd
from collections import defaultdict
import numpy as np

In [12]:

def get_DATA(k, sample_dict, col_name): 
    
    if col_name == 'AADJ2': kor_name = ['자산총계','총자산','銃자산']
    elif col_name == 'AADJ8': kor_name = ['유동자산']
    elif col_name == 'AADJ4': kor_name = ['자본총계','총자본','銃자본']
    elif col_name == 'AADJ6': kor_name = ['유동부채']
    elif col_name == 'AADJ7': kor_name = ['비유동부채', '고정부채']
    elif col_name == 'AADJ5': kor_name = ['매출액']

        
    for kor in kor_name:
        if kor in sample_dict[k]:
            out = sample_dict[k].get(kor)
            break
        else: 
            out = ['','','']

    return out


In [8]:
def cleansing(text):
    pattern = r'quot'  # 일본어,한자제거  quot
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'([ぁ-ゟ゠-ヿ一-鿋])'  # 일본어,한자제거  quot
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'\xa0'  # 줄바꿈제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'URL 복사 이웃추가 본문 기타 기능 번역보기'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'([ㄱ-ㅎㅏ-ㅣ])'  # 한글자음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'[^\w\s]'  # 특수기호 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'[\Ⅰ\Ⅱ\ㆍ]'  # 로마자 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r'[0-9]'  # 로마자 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = r' '  # 띄어쓰기 제거
    text = re.sub(pattern=pattern, repl='', string=text)

    return text

In [15]:
def main():
    
    # 어쩌고 데이터프레임 
    df_ASSET_TRANSFER = pd.read_csv('C:\\Users\\HS\\Desktop\\3,4차 기업이벤트 양수도 전체 목록(결정 삭제).csv', encoding='ANSI')
    df_ASSET_TRANSFER['url'] = df_ASSET_TRANSFER.rcp_no.apply(
        lambda x: "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + str(x))
    df_ASSET_TRANSFER['기재정정'] = df_ASSET_TRANSFER['rpt_nm'].isin(
        ['[기재정정]자산양수ㆍ도종료보고서', '[기재정정]합병등종료보고서(자산양수도)', '[기재정정]합병등종료보고서(영업양수도)', '[기재정정]영업양수ㆍ도종료보고서']).apply(int)
    rpt_nm_list = [
        ['합병등종료보고서(자산양수도)', '[기재정정]합병등종료보고서(자산양수도)'],  # 8,9
        ['합병등종료보고서(영업양수도)', '[기재정정]합병등종료보고서(영업양수도)'],  # 8,9
        ['영업양수ㆍ도종료보고서', '[기재정정]영업양수ㆍ도종료보고서'],  # 4,5
        ['자산양수ㆍ도종료보고서', '[기재정정]자산양수ㆍ도종료보고서']  # 3,4
    ]

    df_sample = df_ASSET_TRANSFER.loc[df_ASSET_TRANSFER['삭제'].isnull()].loc[
        df_ASSET_TRANSFER.rpt_nm.isin(rpt_nm_list[0])]
    #df_sample = df_sample.loc[df_sample.index.isin([0,3,16,17,18])]
    df_sample= df_sample.iloc[:200,:]
    
    
    ###### 돌리기~~~~~
    dart_div_list = list()

    for i in range(0, 100):
        target = df_sample.iloc[i]
        # print(target)
        crpCd = target.crp_cd
        crpNm = target.crp_nm
        rcpNo = target.rcp_no
        rcpDt = target.rcp_dt
        link = target.url

        handle = urllib.request.urlopen(link)
        print('여기다!!!', handle)
        data = handle.read()
        soup2 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')  # http read
        # print(soup2)

        test = soup2.find('a', {'href': '#download'})['onclick']
        # print(test)

        words = test.split("'")
        # print(words)

        rcpNo = words[1]
        dcmNo = words[3]
        # print(rcpNo)
        # print(dcmNo)

        dart2 = soup2.find_all(string=re.compile('dart2.dtd'))
        # print(dart2)
        dart3 = soup2.find_all(string=re.compile('dart3.xsd'))
        # print(dart3)

        if target['기재정정'] == 0:
            eleId = '8'  # 재무제표 8
        else:
            eleId = '9'  # 재무제표 8

            
        ####### 뷰어 링크
        if len(dart3) != 0:
            link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=" + eleId + "&offset=4916&length=3668&dtd=dart3.xsd"
        print(link2)

        
        ####### 테이블 찾기
        handle = urllib.request.urlopen(link2)
        # print(handle)
        data = handle.read()
        soup3 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')
        # print(soup3)

        tables = soup3.findAll("table")  # list of the all tables in the link2 which means number of tables
        print('len(tables):', len(tables))  # length of the list

        
        if len(tables) == 0:
            print('해당사항없음, 판단 -> n_tables 컬럼 ')
            # continue 하게 되면. 누락문제발생
            continue
        elif len(tables) == 1:  # 재무제표만
            div_table = soup3.find("table")
        elif 2 <= len(tables) <= 6:  # 단위, 재무제표
            div_table = soup3.findAll("table")[1]
        else: 
            print('일단 한번 보자, 이후에 이유쓰는 란 추가 ')
            break

        # print(div_table)  # 예외처리 후 한가지의 table 파싱

        
        # rows
        div_trs = div_table.findAll('tr')  # list of the all tr in that tables 
        print('len(div_trs)',len(div_trs))  # the length of the rows
        print(div_trs)
        
        
        # columns -> 4개 ok, 2,5개 예외처리 해야함
        div_tds = div_trs[1]  # list of the all td in that tables
        print('len(div_tds)',len(div_tds))  # the number of columns 
        print(div_tds)
        
        
        
        
        
        ####### 딕셔너리에 넣기
        dict1 = defaultdict(list)

        for i in range(len(div_trs)):
            for j in range(1, 4):
                try:
                    key = cleansing(div_trs[i].findAll('td')[0].text.strip())
                    val = div_trs[i].findAll('td')[j].text.strip()
                    dict1[key].append(val)
                except IndexError:
                    pass
        print(dict(dict1))
        dict1['STK_CD'].append(crpCd)
        dict1['STK_NM_KOR'].append(crpNm)
        dict1['rcp_NO'].append(rcpNo)
        dict1['rcp_DT'].append(rcpDt)
        dict1['url'].append(link)

        dart_div_list.append(dict(dict1))

    print("")
    print("")
    print('dart_div_list')
    print(dart_div_list)

    
    ##### 엑셀에 저장하기 
    cur_dir = os.getcwd()

    workbook_name = "DART_dividends_baseline_1.xlsx"

    workbook = xlsxwriter.Workbook(workbook_name)

    #포멧지정
    worksheet_result = workbook.add_worksheet('result')
    filter_format = workbook.add_format({'bold': True,
                                         'fg_color': '#D7E4BC'
                                         })

    percent_format = workbook.add_format({'num_format': '0.00%'})

    roe_format = workbook.add_format({'bold': True,
                                      'underline': True,
                                      'num_format': '0.00%'})

    num_format = workbook.add_format({'num_format': '0.00'})
    num2_format = workbook.add_format({'num_format': '#,##0'})
    num3_format = workbook.add_format({'num_format': '#,##0.00',
                                       'fg_color': '#FCE4D6'})

    worksheet_result.set_column('A:A', 10)
    worksheet_result.set_column('B:B', 10)
    worksheet_result.set_column('C:C', 10)
    worksheet_result.set_column('D:D', 10)
    worksheet_result.set_column('H:H', 10)
    worksheet_result.set_column('I:I', 10)
    worksheet_result.set_column('J:J', 10)
    worksheet_result.set_column('K:K', 10)
    worksheet_result.set_column('L:L', 10)
    worksheet_result.set_column('M:M', 10)
    worksheet_result.set_column('N:N', 10)
    worksheet_result.set_column('O:O', 10)
    worksheet_result.set_column('P:P', 10)

    worksheet_result.write(0, 0, "STK_CD", filter_format)
    worksheet_result.write(0, 1, "STK_NM_KOR", filter_format)
    worksheet_result.write(0, 2, "rcp_NO", filter_format)
    worksheet_result.write(0, 3, "rcp_DT", filter_format)

    worksheet_result.write(0, 4, "자산_양수도전_자산총계", filter_format)
    worksheet_result.write(0, 5, "자산_증가감소_자산총계", filter_format)
    worksheet_result.write(0, 6, "자산_양수도후_자산총계", filter_format)

    worksheet_result.write(0, 7, "자산_양수도전_유동자산", filter_format)
    worksheet_result.write(0, 8, "자산_증가감소_유동자산", filter_format)
    worksheet_result.write(0, 9, "자산_양수도후_유동자산", filter_format)

    worksheet_result.write(0, 10, "자산_양수도전_자본총계", filter_format)
    worksheet_result.write(0, 11, "자산_증가감소_자본총계", filter_format)
    worksheet_result.write(0, 12, "자산_양수도후_자본총계", filter_format)

    worksheet_result.write(0, 13, "자산_양수도전_유동부채", filter_format)
    worksheet_result.write(0, 14, "자산_증가감소_유동부채", filter_format)
    worksheet_result.write(0, 15, "자산_양수도후_유동부채", filter_format)

    worksheet_result.write(0, 16, "자산_양수도전_비유동부채", filter_format)
    worksheet_result.write(0, 17, "자산_증가감소_비유동부채", filter_format)
    worksheet_result.write(0, 18, "자산_양수도후_비유동부채", filter_format)

    worksheet_result.write(0, 19, "url", filter_format)
    worksheet_result.write(0, 20, "n_tables", filter_format)
    worksheet_result.write(0, 21, "n_trs", filter_format)    
    worksheet_result.write(0, 21, "n_tds", filter_format)    

    
    ##### 데이터 작성하기 
    for k in range(len(dart_div_list)):

        STK_CD = dart_div_list[k].get('STK_CD')[0]  # 기업코드
        STK_NM_KOR = dart_div_list[k].get('STK_NM_KOR')[0]  #기업명
        rcp_NO = dart_div_list[k].get('rcp_NO')[0]  # 보고서번호
        rcp_DT = dart_div_list[k].get('rcp_DT')[0]  # 보고기일
        url = dart_div_list[k].get('url')[0]  # 보고서url

        # 여기 수정 들어감
        AADJ2 = get_DATA(k, dart_div_list, 'AADJ2')
        AADJ8 = get_DATA(k, dart_div_list, 'AADJ8')
        AADJ4 = get_DATA(k, dart_div_list, 'AADJ4')
        AADJ6 = get_DATA(k, dart_div_list, 'AADJ6')
        AADJ7 = get_DATA(k, dart_div_list, 'AADJ7')


        asset_before_AADI2 = AADJ2[0]  # 자산_양수도전_자산총계
        asset_increase_AADI2 = AADJ2[1]  # 자산_증가감소_자산총계
        asset_after_AADJ2 = AADJ2[2]  # 자산_양수도후_자산총계

        asset_before_AADI8 = AADJ8[0]  # 자산_양수도전_유동자산
        asset_increase_AADI8 = AADJ8[1]  # 자산_증가감소_유동자산
        asset_after_AADJ8 = AADJ8[2]  # 자산_양수도후_유동자산

        asset_before_AADI4 = AADJ4[0]  # 자산_양수도전_자본총계
        asset_increase_AADI4 = AADJ4[1]  # 자산_증가감소_자본총계
        asset_after_AADJ4 = AADJ4[2]  # 자산_양수도후_자본총계

        asset_before_AADI6 = AADJ6[0]  # 자산_양수도전_유동부채
        asset_increase_AADI6 = AADJ6[1]  # 자산_증가감소_유동부채
        asset_after_AADJ6 = AADJ6[2]  # 자산_양수도후_유동부채

        asset_before_AADI7 = AADJ7[0]  # 자산_양수도전_비유동부채
        asset_increase_AADI7 = AADJ7[1]  # 자산_증가감소_비유동부채
        asset_after_AADJ7 = AADJ7[2]  # 자산_양수도후_비유동부채


        worksheet_result.write(k + 1, 0, STK_CD)
        worksheet_result.write(k + 1, 1, STK_NM_KOR)
        worksheet_result.write(k + 1, 2, rcp_DT)
        worksheet_result.write(k + 1, 3, rcp_NO)

        worksheet_result.write(k + 1, 4, asset_before_AADI2)
        worksheet_result.write(k + 1, 5, asset_increase_AADI2)
        worksheet_result.write(k + 1, 6, asset_after_AADJ2)

        worksheet_result.write(k + 1, 7, asset_before_AADI8)
        worksheet_result.write(k + 1, 8, asset_increase_AADI8)
        worksheet_result.write(k + 1, 9, asset_after_AADJ8)

        worksheet_result.write(k + 1, 10, asset_before_AADI4)
        worksheet_result.write(k + 1, 11, asset_increase_AADI4)
        worksheet_result.write(k + 1, 12, asset_after_AADJ4)

        worksheet_result.write(k + 1, 13, asset_before_AADI6)
        worksheet_result.write(k + 1, 14, asset_increase_AADI6)
        worksheet_result.write(k + 1, 15, asset_after_AADJ6)

        worksheet_result.write(k + 1, 16, asset_before_AADI7)
        worksheet_result.write(k + 1, 17, asset_increase_AADI7)
        worksheet_result.write(k + 1, 18, asset_after_AADJ7)

        worksheet_result.write(k + 1, 19, url)
        worksheet_result.write(k + 1, 20, len(tables))
        worksheet_result.write(k + 1, 21, len(div_trs))
        worksheet_result.write(k + 1, 22, len(div_tds))
        
    workbook.close()

    
# Main
if __name__ == "__main__":
    start= time.time()
    
    main()
    
    print(time.time()- start)

여기다!!! <http.client.HTTPResponse object at 0x000002321397AB70>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20171113000213&dcmNo=5840458&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 1
len(div_trs) 14
[<tr>
<td align="CENTER" height="20" width="128">구    분</td>
<td align="CENTER" height="20" width="135">자산양수도 전</td>
<td align="CENTER" height="20" width="127">증가(감소)</td>
<td align="CENTER" height="20" width="130">자산양수도 후</td>
</tr>, <tr>
<td height="20" valign="TOP" width="128">자산</td>
<td align="RIGHT" height="20" valign="TOP" width="135"><br/></td>
<td align="RIGHT" height="20" valign="TOP" width="127"><br/></td>
<td align="RIGHT" height="20" valign="TOP" width="130"><br/></td>
</tr>, <tr>
<td height="20" valign="TOP" width="128">Ⅰ.유동자산</td>
<td align="RIGHT" height="20" valign="TOP" width="135">162,257</td>
<td align="RIGHT" height="20" valign="TOP" width="127">+85,350</td>
<td align="RIGHT" height="20" valign="TOP" width="130">247,607</td>
</tr>, <tr>
<td height="20" va

여기다!!! <http.client.HTTPResponse object at 0x000002321397ADD8>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20090521000091&dcmNo=2315033&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 4
len(div_trs) 4
[<tr>
<th align="CENTER" height="40" width="177">구분</th>
<th align="CENTER" height="40" width="141">양수도전<br/>(2008.12.31 기준)</th>
<th align="CENTER" height="40" width="141">증감</th>
<th align="CENTER" height="40" width="141">양수도후</th>
</tr>, <tr>
<td align="CENTER" height="27" valign="TOP" width="177">지분법적용투자주식</td>
<td align="RIGHT" height="27" valign="TOP" width="141">1,022,675</td>
<td align="RIGHT" height="27" valign="TOP" width="141">53,370</td>
<td align="RIGHT" height="27" valign="TOP" width="141">1,076,045</td>
</tr>, <tr>
<td align="CENTER" height="27" valign="TOP" width="177">단기차입금</td>
<td align="RIGHT" height="27" valign="TOP" width="141">117,428</td>
<td align="RIGHT" height="27" valign="TOP" width="141">228,500</td>
<td align="RIGHT" height="27" valign="TOP" width

여기다!!! <http.client.HTTPResponse object at 0x0000023213685898>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20150723000300&dcmNo=4743917&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 1
len(div_trs) 15
[<tr>
<th align="CENTER" height="20" width="201">구 분</th>
<th align="CENTER" height="20" width="128">자산양수전</th>
<th align="CENTER" height="20" width="128">자산양수후</th>
<th align="CENTER" height="20" width="128">증감액</th>
</tr>, <tr>
<td height="20" width="201">자 산</td>
<td height="20" width="128">　</td>
<td height="20" width="128">　</td>
<td height="20" width="128">　</td>
</tr>, <tr>
<td height="20" width="201"> 1. 유동자산</td>
<td align="RIGHT" height="20" width="128">493,947 </td>
<td align="RIGHT" height="20" width="128">493,947 </td>
<td align="RIGHT" height="20" width="128">0 </td>
</tr>, <tr>
<td height="20" width="201"> 2. 비유동자산</td>
<td align="RIGHT" height="20" width="128">1,001,001 </td>
<td align="RIGHT" height="20" width="128">1,113,272 </td>
<td align="RIGHT" height="2

http://dart.fss.or.kr/report/viewer.do?rcpNo=20110902000189&dcmNo=3144356&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 1
len(div_trs) 12
[<tr>
<td height="20" valign="TOP" width="157">항목</td>
<td height="20" valign="TOP" width="156">양도 전</td>
<td height="20" valign="TOP" width="156">증감</td>
<td height="20" valign="TOP" width="155">양도 후</td>
</tr>, <tr>
<td height="20" valign="TOP" width="157">자산</td>
<td align="RIGHT" height="20" valign="TOP" width="156">3,434,082</td>
<td align="RIGHT" height="20" valign="TOP" width="156">(65,606)</td>
<td align="RIGHT" height="20" valign="TOP" width="155">3,368,476 </td>
</tr>, <tr>
<td height="20" valign="TOP" width="157">  유동자산</td>
<td align="RIGHT" height="20" valign="TOP" width="156">481,942</td>
<td align="RIGHT" height="20" valign="TOP" width="156">541,856 </td>
<td align="RIGHT" height="20" valign="TOP" width="155">1,023,798 </td>
</tr>, <tr>
<td height="20" valign="TOP" width="157">  비유동자산</td>
<td align="RIGHT" height="20" val

여기다!!! <http.client.HTTPResponse object at 0x000002321356BEB8>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180920000266&dcmNo=6326191&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 1
len(div_trs) 18
[<tr>
<th align="CENTER" height="20" width="161">구 분</th>
<th align="CENTER" height="20" width="146">자산양수도 전 금액</th>
<th align="CENTER" height="20" width="150">증감액</th>
<th align="CENTER" height="20" width="151">자산양수도 후 금액</th>
</tr>, <tr>
<td height="20" valign="TOP" width="161">자산</td>
<td align="CENTER" height="20" width="146"><br/></td>
<td align="CENTER" height="20" width="150"><br/></td>
<td align="CENTER" height="20" width="151"><br/></td>
</tr>, <tr>
<td height="20" width="161">ㆍ유동자산</td>
<td align="RIGHT" height="20" width="146">43,352,412,334</td>
<td align="RIGHT" height="20" width="150">7,895,900,000</td>
<td align="RIGHT" height="20" width="151">51,248,312,334</td>
</tr>, <tr>
<td height="20" width="161">ㆍ비유동자산</td>
<td align="RIGHT" height="20" width="146">30,38

...

http://dart.fss.or.kr/report/viewer.do?rcpNo=20120330001472&dcmNo=3328557&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 0
해당사항없음, 판단 -> n_tables 컬럼 
여기다!!! <http.client.HTTPResponse object at 0x0000023213B24E80>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20090831000966&dcmNo=2397438&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 0
해당사항없음, 판단 -> n_tables 컬럼 
여기다!!! <http.client.HTTPResponse object at 0x00000232139A2C88>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20151218000494&dcmNo=4890556&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 1
len(div_trs) 2
[<tr>
<td align="CENTER" height="20" width="150">자산목록</td>
<td align="CENTER" height="20" width="150">양수도 전</td>
<td align="CENTER" height="20" width="150">증가(감소)</td>
<td align="CENTER" height="20" width="150">양수도 후</td>
</tr>, <tr>
<td align="RIGHT" height="20" width="150">종속기업투자주식</td>
<td align="RIGHT" height="20" width="150">95,548,553,340</td>
<td align="RIGHT" height="20" width="150">(2,040

http://dart.fss.or.kr/report/viewer.do?rcpNo=20091230000461&dcmNo=2488801&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 1
len(div_trs) 2
[<tr>
<th align="CENTER" height="20" width="150">항  목</th>
<th align="CENTER" height="20" width="151">양수ㆍ도 전</th>
<th align="CENTER" height="20" width="151">증가(감소)</th>
<th align="CENTER" height="20" width="151">양수ㆍ도 후</th>
</tr>, <tr>
<td align="CENTER" height="20" valign="TOP" width="150">유형자산</td>
<td align="RIGHT" height="20" valign="TOP" width="151">4,075</td>
<td align="RIGHT" height="20" valign="TOP" width="151">2,670</td>
<td align="RIGHT" height="20" valign="TOP" width="151">6,745</td>
</tr>]
len(div_tds) 9
<tr>
<td align="CENTER" height="20" valign="TOP" width="150">유형자산</td>
<td align="RIGHT" height="20" valign="TOP" width="151">4,075</td>
<td align="RIGHT" height="20" valign="TOP" width="151">2,670</td>
<td align="RIGHT" height="20" valign="TOP" width="151">6,745</td>
</tr>
{'유형자산': ['4,075', '2,670', '6,745']}
여기다!!! <http.cl

len(tables): 1
len(div_trs) 15
[<tr>
<td align="CENTER" height="20" width="175">구 분</td>
<td align="CENTER" height="20" width="147">자산양수도 전</td>
<td align="CENTER" height="20" width="142">증가(감소)</td>
<td align="CENTER" height="20" width="152">자산양수도 후</td>
</tr>, <tr>
<td height="20" valign="TOP" width="175">[유동자산]</td>
<td align="RIGHT" height="20" width="147">19,350,542,562</td>
<td align="RIGHT" height="20" width="142">-</td>
<td align="RIGHT" height="20" width="152">19,350,542,562</td>
</tr>, <tr>
<td height="20" valign="TOP" width="175">[비유동자산]</td>
<td align="RIGHT" height="20" width="147">98,675,028,133</td>
<td align="RIGHT" height="20" width="142"><br/></td>
<td align="RIGHT" height="20" width="152">98,675,028,133</td>
</tr>, <tr>
<td align="CENTER" height="20" valign="TOP" width="175">자산총계</td>
<td align="RIGHT" height="20" width="147">118,025,570,695</td>
<td align="RIGHT" height="20" width="142">-</td>
<td align="RIGHT" height="20" width="152">118,025,570,695</td>
</tr>, <tr

http://dart.fss.or.kr/report/viewer.do?rcpNo=20101028000112&dcmNo=2804768&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 2
len(div_trs) 2
[<tr>
<th height="20" width="172">구       분</th>
<th height="20" width="131">양수도 전</th>
<th height="20" width="131">증가(감소)</th>
<th height="20" width="130">양수도 후</th>
</tr>, <tr>
<td height="20" width="172">지분법적용투자주식</td>
<td align="RIGHT" height="20" valign="TOP" width="131">6,250</td>
<td align="RIGHT" height="20" valign="TOP" width="131">(5,953)</td>
<td align="RIGHT" height="20" valign="TOP" width="130">297</td>
</tr>]
len(div_tds) 9
<tr>
<td height="20" width="172">지분법적용투자주식</td>
<td align="RIGHT" height="20" valign="TOP" width="131">6,250</td>
<td align="RIGHT" height="20" valign="TOP" width="131">(5,953)</td>
<td align="RIGHT" height="20" valign="TOP" width="130">297</td>
</tr>
{'지분법적용투자주식': ['6,250', '(5,953)', '297']}
여기다!!! <http.client.HTTPResponse object at 0x0000023213B946D8>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20130

12.048915147781372
